<a href="https://colab.research.google.com/github/Aparnamol-KS/VidSafe/blob/Aparna/AudioMasking_Vidsafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pipeline

In [ ]:
!pip install faster-whisper
!pip install transformers
!pip install ffmpeg-python


In [ ]:
!apt-get -y install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


#Extracting Audio from video

In [ ]:
from moviepy.editor import VideoFileClip

# Load your video file
video = VideoFileClip("/content/1000088539.mp4")

# Extract audio
audio = video.audio

# Export audio as MP3
audio.write_audiofile("/content/extracted_audio.mp3", codec='mp3')

print("Audio extracted successfully!")


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



MoviePy - Writing audio in /content/extracted_audio.mp3



chunk:  98%|█████████▊| 1157/1178 [00:01<00:00, 868.97it/s, now=None]
                                                                     

MoviePy - Done.
Audio extracted successfully!


#Audio Generation

In [ ]:
!pip install gTTS



In [ ]:
from gtts import gTTS

# Combine all sentences into a single passage
offensive_sentences = [
    "How are you doing today?",
    "This is a friendly conversation.",
    "You are such an idiot.",
    "Shut the hell up.",
    "The weather today is sunny and beautiful.",
    "I am looking forward to our trip next week.",
    "She bought a new book from the store.",
    "Thank you for helping me with the task.",
    "What the hell are you doing?",
    "This is a stupid idea.",
    "You piece of crap.",
    "This is a friendly conversation.",
    "Don't be a jerk.",
    "Can you help me with my homework?",
    "This project is coming along very nicely.",
    "Please remember to submit your assignment on time.",

    "Stop talking nonsense you fool.",
    "You are acting like an asshole.",
    "Mind your damn business.",
     "Hello, how are you doing today?",
    "Let's meet tomorrow at the cafe.",
    "I really enjoyed the movie last night.",
    "Let's meet tomorrow morning.",
    "This is freaking stupid."
]

# Join them into a single string, adding optional pauses using punctuation
passage = " ".join(offensive_sentences)

# Generate audio
tts = gTTS(text=passage, lang='en')
tts.save("offensive_combined.mp3")

print("Combined offensive audio saved as offensive_combined.mp3")


Combined offensive audio saved as offensive_combined.mp3


# Whisper model

In [ ]:
!pip install faster-whisper


In [ ]:
from faster_whisper import WhisperModel

audio_file = "offensive_combined.mp3"

# Load smaller, faster model
model = WhisperModel("small", device="cuda", compute_type="float16")

# Transcribe audio with word-level timestamps
segments, info = model.transcribe(audio_file, word_timestamps=True)

# Store sentences along with their corresponding words
sentence_segments = []

for seg in segments:
    sentence_data = {
        "text": seg.text.strip(),
        "start": seg.start,
        "end": seg.end,
        "words": []
    }

    for w in seg.words:
        sentence_data["words"].append({
            "word": w.word,
            "start": w.start,
            "end": w.end
        })

    sentence_segments.append(sentence_data)

# Example output
print("Example sentence segment:", sentence_segments[0])
print("Words in this sentence:", sentence_segments[0]["words"])


Example sentence segment: {'text': 'How are you doing today? This is a friendly conversation.', 'start': np.float64(0.0), 'end': np.float64(3.7), 'words': [{'word': ' How', 'start': np.float64(0.0), 'end': np.float64(0.28)}, {'word': ' are', 'start': np.float64(0.28), 'end': np.float64(0.38)}, {'word': ' you', 'start': np.float64(0.38), 'end': np.float64(0.52)}, {'word': ' doing', 'start': np.float64(0.52), 'end': np.float64(0.84)}, {'word': ' today?', 'start': np.float64(0.84), 'end': np.float64(1.24)}, {'word': ' This', 'start': np.float64(1.94), 'end': np.float64(2.18)}, {'word': ' is', 'start': np.float64(2.18), 'end': np.float64(2.36)}, {'word': ' a', 'start': np.float64(2.36), 'end': np.float64(2.5)}, {'word': ' friendly', 'start': np.float64(2.5), 'end': np.float64(2.78)}, {'word': ' conversation.', 'start': np.float64(2.78), 'end': np.float64(3.7)}]}
Words in this sentence: [{'word': ' How', 'start': np.float64(0.0), 'end': np.float64(0.28)}, {'word': ' are', 'start': np.float6

#Detecting Toxicity


In [ ]:
!pip install detoxify


###Sentence level profanity detection

In [ ]:
from detoxify import Detoxify

# Load Detoxify model
detox_model = Detoxify('original')  # you can also use 'multilingual'

# Threshold for classifying toxicity
THRESHOLD = 0.5

# Detect toxic sentences and store the score inside sentence_segments
for seg in sentence_segments:  # sentence_segments contains text + words
    result = detox_model.predict(seg["text"])
    score = float(result["toxicity"])  # probability of being toxic
    seg["toxicity"] = score
    seg["is_toxic"] = score > THRESHOLD

# Extract only toxic sentences if needed
toxic_sentences = [seg for seg in sentence_segments if seg["is_toxic"]]

# Example output
print("Example toxic sentence:", toxic_sentences[0] if toxic_sentences else "None")


Example toxic sentence: {'text': 'You are such an idiot. Shut the hell up. The weather today is sunny and beautiful.', 'start': np.float64(4.32), 'end': np.float64(9.4), 'words': [{'word': ' You', 'start': np.float64(4.32), 'end': np.float64(4.56)}, {'word': ' are', 'start': np.float64(4.56), 'end': np.float64(4.68)}, {'word': ' such', 'start': np.float64(4.68), 'end': np.float64(4.94)}, {'word': ' an', 'start': np.float64(4.94), 'end': np.float64(5.14)}, {'word': ' idiot.', 'start': np.float64(5.14), 'end': np.float64(5.4)}, {'word': ' Shut', 'start': np.float64(6.1), 'end': np.float64(6.26)}, {'word': ' the', 'start': np.float64(6.26), 'end': np.float64(6.44)}, {'word': ' hell', 'start': np.float64(6.44), 'end': np.float64(6.66)}, {'word': ' up.', 'start': np.float64(6.66), 'end': np.float64(6.98)}, {'word': ' The', 'start': np.float64(7.28), 'end': np.float64(7.42)}, {'word': ' weather', 'start': np.float64(7.42), 'end': np.float64(7.7)}, {'word': ' today', 'start': np.float64(7.7),

###Word level profanity detection

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import string

# --------------------------
# 1️⃣ Load Roberta Toxicity Model
# --------------------------
model_name = "s-nlp/roberta_toxicity_classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

THRESHOLD = 0.5  # Adjust for word-level detection (lower if you want more sensitivity)

# --------------------------
# 2️⃣ Helper function to get toxicity score
# --------------------------
def toxic_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=1)[0]
        return float(probs[1])  # probability of toxic class

# --------------------------
# 3️⃣ Word-level toxicity detection
# --------------------------
def get_word_level_toxicity(toxic_sentences, threshold=THRESHOLD):
    masked_segments = []  # List of (start_time, end_time, word, score)

    for seg in toxic_sentences:
        words = seg["words"]
        for w in words:
            word_text = w["word"].strip().lower().translate(str.maketrans("", "", string.punctuation))
            if not word_text:
                continue
            score = toxic_score(word_text)
            if score >= threshold:
                masked_segments.append({
                    "start": float(w["start"]),
                    "end": float(w["end"]),
                    "word": w["word"].strip(),
                    "score": score
                })

    return masked_segments

# --------------------------
# 4️⃣ Run word-level detection
# --------------------------
word_level_toxic = get_word_level_toxicity(toxic_sentences)

# --------------------------
# 5️⃣ Show results
# --------------------------
print("Word-level toxic segments:")
for seg in word_level_toxic:
    print(f"{seg['start']:.2f} → {seg['end']:.2f}  [{seg['word']}]  score: {seg['score']:.3f}")


Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Word-level toxic segments:
5.14 → 5.40  [idiot.]  score: 1.000
20.30 → 20.62  [stupid]  score: 0.999
22.46 → 22.80  [crap.]  score: 0.999
37.32 → 37.68  [fool.]  score: 0.998
39.36 → 39.68  [asshole.]  score: 0.997
40.90 → 41.18  [damn]  score: 0.998
52.62 → 53.12  [stupid.]  score: 0.999


### Beep Voice


In [ ]:
from pydub import AudioSegment
from pydub.generators import Sine

# Load your MP3 audio
audio = AudioSegment.from_file("/content/offensive_combined.mp3", format="mp3")

# Apply mute/beep for each toxic word
for seg in word_level_toxic:
    start_ms = int(seg['start'] * 1000)  # convert to milliseconds
    end_ms = int(seg['end'] * 1000)

    # Option 1: Silence
    # audio = audio[:start_ms] + AudioSegment.silent(duration=(end_ms-start_ms)) + audio[end_ms:]

    # Option 2: Beep instead of silence
    beep = Sine(1000).to_audio_segment(duration=(end_ms-start_ms))
    audio = audio[:start_ms] + beep + audio[end_ms:]

# Save the modified audio
audio.export("censored_audio.mp3", format="mp3")


<_io.BufferedRandom name='censored_audio.mp3'>